In [33]:
#示例：langgraph_hello.py
import os
from dotenv import load_dotenv
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_community.chat_models import ChatZhipuAI
# pip install langgraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
import requests
import json


# 加载 .env 文件中的环境变量
load_dotenv(dotenv_path='D:/02File/05练习项目/04llm_learning/KEYs.env', override=True)
# 获取 API 密钥
llm_api_key = os.environ["ZHIPUAI_API_KEY"] 

llm = ChatZhipuAI(
    model="GLM-4-Air",
    api_key=llm_api_key,
    temperature=0
)

weather_api_key = os.environ["WEATHER_API_KEY"]

def city_code_search(city_name: str) -> str:
    """
    输入城市名称，返回城市代码。
    """
    # 打开并读取JSON文件
    with open("D:/02File/05练习项目/04llm_learning/month1/week1/langgraph/tools/city_code.json", 'r', encoding='utf-8') as file:
        data = json.load(file)
    # 访问数据
    for province in data['list']:
        for city in province['list']:
            if city['name'] == city_name:
                city_id = city['city_id']
    if city_name not in ["北京","天津","上海","重庆"]:
        return city_id+"01"
    else:
        return city_id
    return None
# 定义工具函数，用于代理调用外部工具
@tool
def search(city_name: str) -> str:
    """
    输入城市名称，获取天气信息。
    """
    city_code = city_code_search(city_name)
    if city_code:
        # API 地址
        url = "http://api.yytianqi.com/observe"
        # 可选的查询参数
        params = {
            "city": city_code,
            "key": weather_api_key
        }
        # 发送 GET 请求
        response = requests.get(url, params=params)
        # 检查响应状态码
        if response.status_code == 200:
            # 解析返回的数据（假设是 JSON 格式）
            data = response.json()
            print(f"API Response Data: {data}")
            
            tq = data['data']['tq']
            qw = data['data']['qw']
            return f"查询城市当前天气{tq}，气温{qw}度。"
        else:
            print(f"Error: {response.status_code}")   
    else:
        return "不支持查询该城市的天气"

In [38]:
from datetime import datetime

@tool
def time(query: Annotated[str, "任意输入"]) -> str:
    """
    需要获取当前时间时调用此函数。
    """
    # 获取当前的日期和时间
    now = datetime.now()
    # 格式化日期时间输出
    formatted_time = now.strftime("%Y年%m月%d日，星期%w %H时%M分")
    # 将星期天从“星期0”替换为“星期日”
    formatted_time = formatted_time.replace("星期0", "星期日")
    return formatted_time

In [18]:
time.invoke("ss")

'2025年05月11日，星期日 22时36分'

In [39]:
# 定义工具函数，用于代理调用外部工具
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_api_key = os.environ["TAVILY_API_KEY"]
# 创建TavilySearchResults工具，设置最大结果数为1
tools = [TavilySearchResults(max_results=1, tavily_api_key=tavily_api_key), search, time]

In [40]:
from langgraph.prebuilt import create_react_agent

# 从LangChain的Hub中获取prompt模板，可以进行修改
# prompt = hub.pull("wfh/react-agent-executor")
# prompt.pretty_print()

# 创建一个REACT代理执行器，使用指定的LLM和工具，并应用从Hub中获取的prompt
agent_executor = create_react_agent(llm, tools)

In [41]:
agent_executor.invoke({"messages": [("user", "查找2024年巴黎奥运会100米自由泳决赛冠军")]})

{'messages': [HumanMessage(content='查找2024年巴黎奥运会100米自由泳决赛冠军', additional_kwargs={}, response_metadata={}, id='688fce1d-2424-4599-a31e-a40821e2ac03'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query": "2024年巴黎奥运会100米自由泳决赛冠军"}', 'name': 'tavily_search_results_json'}, 'id': 'call_-8724726023226009934', 'index': 0, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 385, 'total_tokens': 409}, 'model_name': 'GLM-4-Air', 'finish_reason': 'tool_calls'}, id='run-7c8d5dfe-0983-4b2a-8d89-96baf82adb7b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024年巴黎奥运会100米自由泳决赛冠军'}, 'id': 'call_-8724726023226009934', 'type': 'tool_call'}]),
  ToolMessage(content='[{"title": "男子100米自由泳决赛：中国选手潘展乐夺冠_2024巴黎奥运会 - 新华网", "url": "http://www.news.cn/sports/20240801/bf63660fb33d4751be87d0b4b445d593/c.html", "content": "7月31日，冠军中国选手潘展乐（中）、亚军澳大利亚选手查默斯（左）与季军罗马尼亚选手波波维奇在颁奖仪式上合影。新华社记者 雒圆 摄\\n\\n\\n\\n7月3

In [42]:
agent_executor.invoke({"messages": [("user", "查询现在的时间")]})

{'messages': [HumanMessage(content='查询现在的时间', additional_kwargs={}, response_metadata={}, id='53f4fc98-502e-47e2-a2fe-7644e3e6e0ca'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query": "现在的时间是多少"}', 'name': 'time'}, 'id': 'call_-8724720525666815989', 'index': 0, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 376, 'total_tokens': 387}, 'model_name': 'GLM-4-Air', 'finish_reason': 'tool_calls'}, id='run-31280dd4-b303-4ac1-a026-58dec7056978-0', tool_calls=[{'name': 'time', 'args': {'query': '现在的时间是多少'}, 'id': 'call_-8724720525666815989', 'type': 'tool_call'}]),
  ToolMessage(content='2025年05月11日，星期日 22时52分', name='time', id='179aba6b-d061-4d37-be96-8f7650e47e17', tool_call_id='call_-8724720525666815989'),
  AIMessage(content='当前时间是2025年5月11日，星期日，晚上10点52分。', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 395, 'total_tokens': 416}, 'model_name'

In [43]:
agent_executor.invoke({"messages": [("user", "大连天气怎么样")]})

API Response Data: {'code': 1, 'msg': 'Sucess', 'counts': 13, 'data': {'cityId': 'CH070201', 'cityName': '大连', 'lastUpdate': '2025-05-11 22:44:22', 'tq': '晴', 'numtq': '00', 'qw': '16', 'numfl': 2, 'fl': '4-5级', 'fx': '南风', 'numfx': '4', 'sd': '55'}}


{'messages': [HumanMessage(content='大连天气怎么样', additional_kwargs={}, response_metadata={}, id='c7fb468b-6f20-40ad-9717-ec43508cd647'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"city_name": "大连"}', 'name': 'search'}, 'id': 'call_-8724728325328989224', 'index': 0, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 376, 'total_tokens': 386}, 'model_name': 'GLM-4-Air', 'finish_reason': 'tool_calls'}, id='run-6e95ab94-4fbe-4094-b201-e92edd6e3c71-0', tool_calls=[{'name': 'search', 'args': {'city_name': '大连'}, 'id': 'call_-8724728325328989224', 'type': 'tool_call'}]),
  ToolMessage(content='查询城市当前天气晴，气温16度。', name='search', id='4bd29015-2174-4647-95a6-27ff989606c2', tool_call_id='call_-8724728325328989224'),
  AIMessage(content='当前大连的天气晴朗，气温为16摄氏度。如果您需要更详细的天气信息，比如风力、湿度等，请告诉我，我可以为您提供。', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 390, 'total_token

In [72]:
# 定义函数，决定是否继续执行
def should_continue(state: MessagesState) -> Literal["tools", END]: # type: ignore
    messages = state['messages']
    last_message = messages[-1]
    # 如果LLM调用了工具，则转到“tools”节点
    if last_message.tool_calls:
        return "tools"
    # 否则，停止（回复用户）
    return END


# 定义调用模型的函数
def call_model(state: MessagesState):
    messages = state['messages']
    # print(messages)
    response = agent_executor.invoke({"messages": messages})
    # 返回列表，因为这将被添加到现有列表中
    return response

In [73]:
# 创建工具节点
tool_node = ToolNode(tools)

In [74]:
# 2.用状态初始化图，定义一个新的状态图
workflow = StateGraph(MessagesState)
# 3.定义图节点，定义我们将循环的两个节点
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# 4.定义入口点和图边
# 设置入口点为“agent”
# 这意味着这是第一个被调用的节点
workflow.set_entry_point("agent")

In [75]:
# 添加条件边
workflow.add_conditional_edges(
    # 首先，定义起始节点。我们使用`agent`。
    # 这意味着这些边是在调用`agent`节点后采取的。
    "agent",
    # 接下来，传递决定下一个调用节点的函数。
    should_continue,
)

# 添加从`tools`到`agent`的普通边。
# 这意味着在调用`tools`后，接下来调用`agent`节点。
workflow.add_edge("tools", 'agent')

# 初始化内存以在图运行之间持久化状态
checkpointer = MemorySaver()

In [76]:
# 5.编译图
# 这将其编译成一个LangChain可运行对象，
# 这意味着你可以像使用其他可运行对象一样使用它。
# 注意，我们（可选地）在编译图时传递内存
app = workflow.compile(checkpointer=checkpointer)
# app = workflow.compile()

In [77]:
# 6.执行图，使用可运行对象
final_state = app.invoke(
    {"messages": [HumanMessage(content="大连的天气怎么样?")]},
    config={"configurable": {"thread_id": 42}}
)
# 从 final_state 中获取最后一条消息的内容
result = final_state["messages"][-1].content
print(result)
print(final_state)

API Response Data: {'code': 1, 'msg': 'Sucess', 'counts': 28, 'data': {'cityId': 'CH070201', 'cityName': '大连', 'lastUpdate': '2025-05-11 23:34:06', 'tq': '晴', 'numtq': '00', 'qw': '15', 'numfl': 3, 'fl': '5-6级', 'fx': '西南风', 'numfx': '5', 'sd': '57'}}
当前大连的天气晴朗，气温为15摄氏度。如果您需要更详细的天气信息，比如风力、湿度等，请告诉我，我可以为您提供。
{'messages': [HumanMessage(content='大连的天气怎么样?', additional_kwargs={}, response_metadata={}, id='1fbb5b01-0e56-4d65-b01e-4903e00e6aa2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"city_name": "大连"}', 'name': 'search'}, 'id': 'call_-8724726298104037640', 'index': 0, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 378, 'total_tokens': 388}, 'model_name': 'GLM-4-Air', 'finish_reason': 'tool_calls'}, id='run-98d3fa36-8be7-4536-ac6c-66e361b2d3f9-0', tool_calls=[{'name': 'search', 'args': {'city_name': '大连'}, 'id': 'call_-8724726298104037640', 'type': 'tool_call'}]), ToolMessage(content='查询城市当前天气晴

In [78]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="查询特朗普的生日")]},
    config={"configurable": {"thread_id": 42}}
)
# 从 final_state 中获取最后一条消息的内容
result = final_state["messages"][-1].content
print(result)
print(final_state)

特朗普的生日是1946年6月14日。
{'messages': [HumanMessage(content='大连的天气怎么样?', additional_kwargs={}, response_metadata={}, id='1fbb5b01-0e56-4d65-b01e-4903e00e6aa2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"city_name": "大连"}', 'name': 'search'}, 'id': 'call_-8724726298104037640', 'index': 0, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 378, 'total_tokens': 388}, 'model_name': 'GLM-4-Air', 'finish_reason': 'tool_calls'}, id='run-98d3fa36-8be7-4536-ac6c-66e361b2d3f9-0', tool_calls=[{'name': 'search', 'args': {'city_name': '大连'}, 'id': 'call_-8724726298104037640', 'type': 'tool_call'}]), ToolMessage(content='查询城市当前天气晴，气温15度。', name='search', id='bf7c60ab-9201-4f30-bbfc-d81c9d0982da', tool_call_id='call_-8724726298104037640'), AIMessage(content='当前大连的天气晴朗，气温为15摄氏度。如果您需要更详细的天气信息，比如风力、湿度等，请告诉我，我可以为您提供。', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 39

In [79]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="我问了哪个城市的天气")]},
    config={"configurable": {"thread_id": 42}}
)
# 从 final_state 中获取最后一条消息的内容
result = final_state["messages"][-1].content
print(result)
print(final_state)

您询问了大连的天气情况。
{'messages': [HumanMessage(content='大连的天气怎么样?', additional_kwargs={}, response_metadata={}, id='1fbb5b01-0e56-4d65-b01e-4903e00e6aa2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"city_name": "大连"}', 'name': 'search'}, 'id': 'call_-8724726298104037640', 'index': 0, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 378, 'total_tokens': 388}, 'model_name': 'GLM-4-Air', 'finish_reason': 'tool_calls'}, id='run-98d3fa36-8be7-4536-ac6c-66e361b2d3f9-0', tool_calls=[{'name': 'search', 'args': {'city_name': '大连'}, 'id': 'call_-8724726298104037640', 'type': 'tool_call'}]), ToolMessage(content='查询城市当前天气晴，气温15度。', name='search', id='bf7c60ab-9201-4f30-bbfc-d81c9d0982da', tool_call_id='call_-8724726298104037640'), AIMessage(content='当前大连的天气晴朗，气温为15摄氏度。如果您需要更详细的天气信息，比如风力、湿度等，请告诉我，我可以为您提供。', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 392, 'to

In [80]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="查询当前时间")]},
    config={"configurable": {"thread_id": 42}}
)
# 从 final_state 中获取最后一条消息的内容
result = final_state["messages"][-1].content
print(result)
print(final_state)

当前时间是2025年5月14日，星期三，07时56分。
{'messages': [HumanMessage(content='大连的天气怎么样?', additional_kwargs={}, response_metadata={}, id='1fbb5b01-0e56-4d65-b01e-4903e00e6aa2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"city_name": "大连"}', 'name': 'search'}, 'id': 'call_-8724726298104037640', 'index': 0, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 378, 'total_tokens': 388}, 'model_name': 'GLM-4-Air', 'finish_reason': 'tool_calls'}, id='run-98d3fa36-8be7-4536-ac6c-66e361b2d3f9-0', tool_calls=[{'name': 'search', 'args': {'city_name': '大连'}, 'id': 'call_-8724726298104037640', 'type': 'tool_call'}]), ToolMessage(content='查询城市当前天气晴，气温15度。', name='search', id='bf7c60ab-9201-4f30-bbfc-d81c9d0982da', tool_call_id='call_-8724726298104037640'), AIMessage(content='当前大连的天气晴朗，气温为15摄氏度。如果您需要更详细的天气信息，比如风力、湿度等，请告诉我，我可以为您提供。', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_to